In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, roc_curve, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.multiclass import OneVsRestClassifier
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.patches as mpatches
from scipy.stats import mode
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
import tensorflow as tf
from sklearn import tree
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.calibration import calibration_curve
from sklearn.linear_model import SGDClassifier
from mpl_toolkits.mplot3d import Axes3D
from statsmodels.tsa.arima.model import ARIMA
# Set pandas to display all columns
# pd.set_option('display.max_columns', None)

**Download Datasets**

In [3]:
# Voice measuement Dataset
# df1 = pd.read_csv("/parkinsons.data")

# Read your five mjf dataframes
mjf1 = pd.read_csv("/content/drive/MyDrive/Mile Stone 2/Dataset/MDS/MDS-UPDRS_Part_I_Patient_Questionnaire_11Sep2024.csv")
mjf2 = pd.read_csv("/content/drive/MyDrive/Mile Stone 2/Dataset/MDS/MDS_UPDRS_Part_II__Patient_Questionnaire_11Sep2024.csv")
mjf3 = pd.read_csv("/content/drive/MyDrive/Mile Stone 2/Dataset/MDS/MDS-UPDRS_Part_III_11Sep2024.csv")

# Demographic and status
demo = pd.read_csv("/content/drive/MyDrive/Mile Stone 2/Dataset/Demographic/Demographics_17Sep2024.csv")
status = pd.read_csv("/content/drive/MyDrive/Mile Stone 2/Dataset/Demographic/Participant_Status_17Sep2024.csv")

# Datadict
data_dict = pd.read_csv("/content/drive/MyDrive/Mile Stone 2/Dataset/MDS/ppmi_data_dictionary_merge.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Mile Stone 2/Dataset/MDS/MDS-UPDRS_Part_I_Patient_Questionnaire_11Sep2024.csv'

In [ ]:
# Concatenate the dataframes along the column axis (axis=1)
concatenated_df = pd.concat([mjf1, mjf2, mjf3], axis=1)

# Remove duplicate 'PATNO', 'REC_ID', 'EVENT_ID' columns if they exist
concatenated_df = concatenated_df.loc[:,~concatenated_df.columns.duplicated()]

# List of columns to retain
columns_to_keep = ['PATNO', 'REC_ID', 'EVENT_ID',
                   'NP1SLPN', 'NP1SLPD', 'NP1PAIN', 'NP1URIN', 'NP1CNST', 'NP1LTHD', 'NP1FATG',
                   'NP2SPCH', 'NP2SALV', 'NP2SWAL', 'NP2EAT', 'NP2DRES', 'NP2HYGN', 'NP2HWRT',
                   'NP2HOBB', 'NP2TURN', 'NP2TRMR', 'NP2RISE', 'NP2WALK', 'NP2FREZ',
                   'NP3SPCH', 'NP3FACXP', 'NP3RIGN', 'NP3RIGRU', 'NP3RIGLU', 'NP3RIGRL', 'NP3RIGLL',
                   'NP3FTAPR', 'NP3FTAPL', 'NP3HMOVR', 'NP3HMOVL', 'NP3PRSPR', 'NP3PRSPL', 'NP3TTAPR',
                   'NP3TTAPL', 'NP3LGAGR', 'NP3LGAGL', 'NP3RISNG', 'NP3GAIT', 'NP3FRZGT', 'NP3PSTBL',
                   'NP3POSTR', 'NP3BRADY', 'NP3PTRMR', 'NP3PTRML', 'NP3KTRMR', 'NP3KTRML',
                   'NP3RTARU', 'NP3RTALU', 'NP3RTARL', 'NP3RTALL', 'NP3RTALJ', 'NP3RTCON']

# Filter the dataframe to include only the specified columns
final_df = concatenated_df[columns_to_keep]

In [ ]:
final_df = final_df.dropna()

# Define the list of event IDs you want to keep (up to V10)
valid_event_ids = ["BL", "V04", "V06", "V10"]

# Filter the dataframe to keep only rows where 'EVENT_ID' is in the valid_event_ids
filtered_until_v10_df = final_df[final_df['EVENT_ID'].isin(valid_event_ids)]

# Save the filtered dataframe to a new CSV file
filtered_until_v10_df.to_csv("filtered_parkinson_patients_until_v10.csv", index=False)

# Confirm the file is saved
print("File saved as 'filtered_parkinson_patients_until_v10.csv'")

In [ ]:
df_cleaned = filtered_until_v10_df[(filtered_until_v10_df != 101).all(axis=1)]

In [ ]:
# First, select the columns of interest from demo and status
demo_selected = demo[['PATNO', 'SEX']]
status_selected = status[['PATNO', 'COHORT_DEFINITION']]

# Step 1: Merge demo with df_cleaned on PATNO
merged_df = pd.merge(df_cleaned, demo_selected, on='PATNO', how='inner')

# Step 2: Merge the result with status on PATNO
final_df = pd.merge(merged_df, status_selected, on='PATNO', how='inner')

# Display the final dataframe
final_df.head()

# Save the final dataframe
final_df.to_csv("merged_parkinson_data.csv", index=False)

# Confirm the file is saved
print("File saved as 'merged_parkinson_data.csv'")

In [ ]:
# Step 1: Create a dictionary to map ITM_NAME to DSCR. Rename the columns to make them more legible
column_mapping = dict(zip(data_dict['ITM_NAME'], data_dict['DSCR']))

# Step 2: Rename columns in final_df based on the mapping
final_df_renamed = final_df.rename(columns=column_mapping)
final_df_renamed.to_csv("final_df_renamed.csv", index=False)

In [ ]:
df = final_df_renamed

categories = {
    'Rigidity': ['3.3a Rigidity - Neck', '3.3b Rigidity - RUE', '3.3c Rigidity - LUE',
                 '3.3d Rigidity - RLE', '3.3e Rigidity - LLE'],
    'Tremor': ['3.15a Postural tremor - Right Hand', '3.15b Postural tremor - Left hand',
               '3.16a Kinetic tremor - Right hand', '3.16b Kinetic tremor - Left hand',
               '3.17a Rest tremor amplitude - RUE', '3.17b Rest tremor amplitude - LUE',
               '3.17c Rest tremor amplitude - RLE', '3.17d Rest tremor amplitude - LLE',
               '3.17e Rest tremor amplitude - Lip/jaw'],
    'Motor Function': ['3.4a Finger Tapping Right Hand', '3.4b Finger Tapping Left Hand',
                       '3.5a Hand movements - Right Hand', '3.5b Hand movements - Left Hand',
                       '3.6a Pronation-Supination - Right Hand', '3.6b Pronation-Supination - Left Hand',
                       '3.7a Toe tapping - Right foot', '3.7b Toe tapping - Left foot',
                       '3.8a Leg agility - Right leg', '3.8b Leg agility - Left leg'],
    'Posture and Gait': ['3.9 Arising from chair', '3.10 Gait', '3.11 Freezing of gait',
                         '3.12 Postural stability', '3.13 Posture'],
    'Non-Motor Symptoms': ['DAYTIME SLEEPINESS', 'PAIN AND OTHER SENSATIONS', 'URINARY PROBLEMS',
                           'CONSTIPATION PROBLEMS', 'LIGHTHEADEDNESS ON STANDING', 'FATIGUE',
                           'SPEECH', 'SALIVA + DROOLING', 'CHEWING AND SWALLOWING', 'EATING TASKS',
                           'DRESSING', 'HYGIENE', 'HANDWRITING', 'DOING HOBBIES AND OTHER ACTIVITIES',
                           'TURNING IN BED']
}

# Create new columns for the sum values of each category
for category, symptoms_list in categories.items():
    df[category + ' Sum'] = df[symptoms_list].sum(axis=1)

for category, symptoms_list in categories.items():
    df[category + ' Mean'] = df[symptoms_list].mean(axis=1)

df_sum = df[['Participant ID','Visit ID','Decoded Value for COHORT',
             'Rigidity Mean', 'Tremor Mean', 'Motor Function Mean', 'Posture and Gait Mean', 'Non-Motor Symptoms Mean']]
df_sum = df[df["Decoded Value for COHORT"] != "SWEDD"]
output_path = r"D:\Downloads\sum_df.csv"
df_sum.to_csv(output_path, index=False)

df_sum = df_sum.drop_duplicates(subset='Participant ID', keep='last').reset_index(drop = True)
df_sum.columns

In [ ]:
df_model = df_sum.drop(columns = ['3.3a Rigidity - Neck', '3.3b Rigidity - RUE', '3.3c Rigidity - LUE',
                 '3.3d Rigidity - RLE', '3.3e Rigidity - LLE', '3.15a Postural tremor - Right Hand', '3.15b Postural tremor - Left hand',
               '3.16a Kinetic tremor - Right hand', '3.16b Kinetic tremor - Left hand',
               '3.17a Rest tremor amplitude - RUE', '3.17b Rest tremor amplitude - LUE',
               '3.17c Rest tremor amplitude - RLE', '3.17d Rest tremor amplitude - LLE',
               '3.17e Rest tremor amplitude - Lip/jaw', '3.4a Finger Tapping Right Hand', '3.4b Finger Tapping Left Hand',
                       '3.5a Hand movements - Right Hand', '3.5b Hand movements - Left Hand',
                       '3.6a Pronation-Supination - Right Hand', '3.6b Pronation-Supination - Left Hand',
                       '3.7a Toe tapping - Right foot', '3.7b Toe tapping - Left foot',
                       '3.8a Leg agility - Right leg', '3.8b Leg agility - Left leg', '3.9 Arising from chair', '3.10 Gait', '3.11 Freezing of gait',
                         '3.12 Postural stability', '3.13 Posture', 'DAYTIME SLEEPINESS', 'PAIN AND OTHER SENSATIONS', 'URINARY PROBLEMS',
                           'CONSTIPATION PROBLEMS', 'LIGHTHEADEDNESS ON STANDING', 'FATIGUE',
                           'SPEECH', 'SALIVA + DROOLING', 'CHEWING AND SWALLOWING', 'EATING TASKS',
                           'DRESSING', 'HYGIENE', 'HANDWRITING', 'DOING HOBBIES AND OTHER ACTIVITIES',
                           'TURNING IN BED'])

In [ ]:
df_model['Decoded Value for COHORT'].value_counts()

In [ ]:
from sklearn.preprocessing import StandardScaler

# Step 1: Preprocessing the data
# Select only numeric columns for PCA
numeric_df = df_model.select_dtypes(include=['float64', 'int64'])
# Remove identifier columns ()
numeric_df = numeric_df.drop(columns=['Participant ID','Sex of participant at birth'], errors='ignore')

# Handle missing values by filling them (you can choose to drop rows/columns with missing data)
numeric_df = numeric_df.fillna(numeric_df.mean())
# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(numeric_df)

# Step 2: Perform PCA
pca = PCA(n_components=0.95)  # Retain 95% of variance
pca_components = pca.fit_transform(scaled_data)

# Step 3: Visualize the explained variance
plt.figure(figsize=(10, 7))
plt.bar(range(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_, alpha=0.5, align='center')
plt.ylabel('Explained Variance Ratio')
plt.xlabel('Principal Components')
plt.title('PCA Explained Variance')
plt.show()

# Step 4: Show feature contributions to the principal components
# Loadings: contributions of each original feature to the principal components
loadings = pd.DataFrame(pca.components_.T, columns=[f'PC{i+1}' for i in range(pca.n_components_)], index=numeric_df.columns)

# Show the loadings for the first few principal components
loadings

In [ ]:
# pip install umap-learn

In [ ]:
# import umap
# import seaborn as sns

# # Create the UMAP reducer
# reducer = umap.UMAP(n_components=2)

# # Fit and transform your data (X is your dataset)
# X_umap = reducer.fit_transform(scaled_data)

# plt.figure(figsize=(8, 6))
# sns.scatterplot(x=X_umap[:, 0], y=X_umap[:, 1], s=50, alpha=0.7, color="blue")
# plt.title("UMAP Projection (2D)")
# plt.xlabel("UMAP 1")
# plt.ylabel("UMAP 2")
# plt.show()




In [ ]:

#according to the chart 95% of the variance in the data was represented by the first 4 principal components

# Use the first 3 principal components for clustering
pca_6_components = pca_components[:, :4]

# Step 1: Apply K-means clustering
kmeans = KMeans(n_clusters=3 , random_state=42)
kmeans.fit(pca_6_components)

# Step 2: Add the cluster labels to the dataset
df_model['cluster'] = kmeans.labels_

# Step 3: Visualize the clustering (PCA components)
plt.figure(figsize=(10, 7))
plt.scatter(pca_6_components[:, 0], pca_6_components[:, 1], c=kmeans.labels_, cmap='viridis')
plt.title('K-means Clustering on PCA-transformed Data')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.colorbar()
plt.show()

In [ ]:
df_model

**DecisionTree**

In [ ]:
# Check if the columns exist before dropping them and handle missing values
columns_to_drop = ["Participant ID", "Visit ID", "Record ID"]

# Drop the columns if they exist, and drop any rows with missing values
df_model2 = df_model.drop(columns=[col for col in columns_to_drop if col in df_sum.columns]).dropna()
# Separate features (X) and target (y)
X_filtered = df_model2.drop(columns=["Decoded Value for COHORT"])
y_filtered = df_model2["Decoded Value for COHORT"]
label_encoder = LabelEncoder()
# Re-encode the target variable
y_filtered_encoded = label_encoder.fit_transform(y_filtered)

# Split the filtered data into training and testing sets
X_train_filtered, X_test_filtered, y_train_filtered, y_test_filtered = train_test_split(
    X_filtered, y_filtered_encoded, test_size=0.2, random_state=42
)

# Train the decision tree model with the filtered data
decision_tree_adjusted_sum = DecisionTreeClassifier(random_state=42, max_depth=5)
decision_tree_adjusted_sum.fit(X_train_filtered, y_train_filtered)


# Adjusting the decision tree visualization to improve readability by reducing font size and controlling layout
plt.figure(figsize=(50, 20))
tree.plot_tree(
    decision_tree_adjusted_sum,
    feature_names=X_filtered.columns,
    class_names=label_encoder.classes_,
    filled=True,
    fontsize=10,  # Smaller font size for readability
    proportion=True,  # Scale boxes based on samples in each node
)
plt.title("Decision Tree for Parkinson's Disease Dataset")
plt.show()

In [ ]:

# Make predictions on the test set
y_pred_filtered = decision_tree_adjusted_sum.predict(X_test_filtered)

# Calculate accuracy, F1 score, and recall
accuracy = accuracy_score(y_test_filtered, y_pred_filtered)
f1 = f1_score(y_test_filtered, y_pred_filtered, average='weighted')
recall = recall_score(y_test_filtered, y_pred_filtered, average='weighted')

# Print the results
print(f'Accuracy: {accuracy}')
print(f'F1 Score (weighted): {f1}')
print(f'Recall (weighted): {recall}')

## Logistic Regression

In [ ]:
df_model.columns

In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/Mile Stone 2/Dataset/MDS/sum_df.csv')
# Proper selection of multiple columns
df = df_model[['Participant ID', 'Record ID', 'Visit ID', 'Sex of participant at birth',
         'Decoded Value for COHORT',
               'Rigidity Mean', 'Tremor Mean', 'Motor Function Mean', 'Posture and Gait Mean', 'Non-Motor Symptoms Mean']]
df

In [ ]:
# Split the data in to 80/20 train test set
X = df.drop(columns=['Participant ID', 'Decoded Value for COHORT','Visit ID','Record ID'])
y = df['Decoded Value for COHORT']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state=42,stratify=y)
y_train = y_train.replace({'Healthy Control': 0, 'Parkinson\'s Disease': 1, 'Prodromal': 0})
y_test = y_test.replace({'Healthy Control': 0, 'Parkinson\'s Disease': 1, 'Prodromal': 0})
print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

In [ ]:
# Identify continuous columns
continuous_cols = X_train.select_dtypes(include=['float64','int64']).columns
# Create preprocessor for continuous columns
preprocessor = ColumnTransformer(
    transformers = [
        ('num',Pipeline(steps =[
            ('imputer',SimpleImputer(strategy='mean')),
            ('scaler',MinMaxScaler())
        ]),continuous_cols)
    ],remainder = 'passthrough'
)
preprocessor

In [ ]:
log_reg = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=42))
])
# Perform cross-validation to compare AUC
log_reg_auc = cross_val_score(log_reg, X_train, y_train, cv=5, scoring='roc_auc')
print(np.std(log_reg_auc))
print(np.mean(log_reg_auc))

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_score, recall_score

y_pred = cross_val_predict(log_reg, X_train, y_train, cv = 5)
precision = precision_score(y_train, y_pred)
recall = recall_score(y_train, y_pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

In [ ]:
#Perform cross-validation and store AUC for Logistic Regression
log_reg.fit(X_train, y_train)  # Train the Logistic Regression model
y_pred_probs_log = log_reg.predict_proba(X_test)[:, 1]  # Get predicted probabilities for ROC and calibration

# Calculate AUC for Logistic Regression
log_reg_auc = roc_auc_score(y_test, y_pred_probs_log)
print(f"Logistic Regression AUC: {log_reg_auc:.3f}")

# Plot ROC Curve for Logistic Regression
fpr_log, tpr_log, _ = roc_curve(y_test, y_pred_probs_log)
plt.figure(figsize=(8, 6))
plt.plot(fpr_log, tpr_log, label=f"Logistic Regression (AUC = {log_reg_auc:.3f})")
plt.plot([0, 1], [0, 1], "k--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - Logistic Regression")
plt.legend()
plt.show()

# Plot Calibration Curve for Logistic Regression
fraction_of_positives_log, mean_predicted_value_log = calibration_curve(y_test, y_pred_probs_log, n_bins=10)

plt.figure(figsize=(8, 6))
plt.plot(mean_predicted_value_log, fraction_of_positives_log, "s-", label="Logistic Regression")
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
plt.title("Calibration Curve - Logistic Regression")
plt.xlabel("Mean Predicted Probability")
plt.ylabel("Fraction of Positives")
plt.legend()
plt.show()

## Random Forest

In [ ]:
rf_clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])
# Perform cross-validation to compare AUC
rf_auc = cross_val_score(rf_clf, X_train, y_train, cv=5, scoring='roc_auc')
print(np.std(rf_auc))
print(np.mean(rf_auc))

In [ ]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_train)

best_model = LogisticRegression(random_state = 42)
best_model.fit(X_scaled, y_train)

coefficients = best_model.coef_[0]
feature_names = X.columns

# Create a DataFrame to pair features with coefficients
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients
})

# Sort by absolute value of the coefficients
importance_df['Importance'] = np.abs(importance_df['Coefficient'])
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Display the top features
print(importance_df)

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'], color='b')
plt.xlabel('Absolute Coefficient Value')
plt.title('Feature Importance in Logistic Regression')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
reg_values = np.logspace(-4, 4, 10)

scores = []

for C in reg_values:
    lr = LogisticRegression(C = C, random_state = 42)
    score = cross_val_score(lr, X_train, y_train, cv=5, scoring='accuracy')
    scores.append(score.mean())

plt.plot(reg_values, scores)
plt.xscale('log')
plt.xlabel('Regularization Strength (C)')
plt.ylabel('Cross-Validated Accuracy')
plt.title('Hyperparameter Sensitivity: Regularization Strength (C)')
plt.show()

In [ ]:
# Perform cross-validation and store AUC for Random Forest
rf_clf.fit(X_train, y_train)  # Train the Random Forest model
y_pred_probs_rf = rf_clf.predict_proba(X_test)[:, 1]  # Get predicted probabilities for ROC and calibration

# Calculate AUC for Random Forest
rf_auc = roc_auc_score(y_test, y_pred_probs_rf)
print(f"Random Forest AUC: {rf_auc:.3f}")

# Plot ROC Curve for Random Forest
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_probs_rf)
plt.figure(figsize=(8, 6))
plt.plot(fpr_rf, tpr_rf, label=f"Random Forest (AUC = {rf_auc:.3f})")
plt.plot([0, 1], [0, 1], "k--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - Random Forest")
plt.legend()
plt.show()

# Plot Calibration Curve for Random Forest
fraction_of_positives_rf, mean_predicted_value_rf = calibration_curve(y_test, y_pred_probs_rf, n_bins=10)

plt.figure(figsize=(8, 6))
plt.plot(mean_predicted_value_rf, fraction_of_positives_rf, "s-", label="Random Forest")
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
plt.title("Calibration Curve - Random Forest")
plt.xlabel("Mean Predicted Probability")
plt.ylabel("Fraction of Positives")
plt.legend()
plt.show()

In [ ]:
best_model = log_reg if log_reg_auc > rf_auc else rf_clf
# Fit the best model to the training data
best_model.fit(X_train, y_train)

# Predict probabilities on the test set
y_pred_prob = best_model.predict_proba(X_test)[:, 1]

# Calculate and report AUC on the test set
test_auc = roc_auc_score(y_test, y_pred_prob)
print(f"Test AUC: {test_auc}")

In [ ]:
plt.figure(figsize=(8, 6))
plt.hist(y_pred_prob, bins=20, edgecolor='k', alpha=0.7)
plt.xlabel('Predicted Probability of Failure')
plt.ylabel('Frequency')
plt.title('Distribution of Predicted Probabilities')
plt.show()

## Identify predicted probability threshold to identify the top 20% highest risk patients

In [ ]:
threshold = np.percentile(y_pred_prob, 80)
print(f"Probability threshold for top 20% highest risk patients: {threshold}")

# Identify students in the top 20% of risk based on this threshold
high_risk_patients = X_test[y_pred_prob >= threshold]
print(f"Number of high-risk patients: {len(high_risk_patients)}")

## Visualization
ROC Curve:
Show how well the model distinguishes between high-risk and low-risk patients for conversion to Parkinson’s.

Calibration Curve:
Demonstrate how reliable the model’s predicted probabilities are for actual patient outcomes.

Distribution of Probabilities:
Use a histogram to show the range of predicted risk probabilities among patients, illustrating the threshold that marks the top 20%.

 ## Key Finding
Highlight the most important findings:

Probability Threshold: The model determined that patients with a predicted risk probability of 0.78 or higher are in the top 20% of those at highest risk for developing Parkinson's disease.

High-Risk Group: Out of the patients in the test set, 275 patients were identified as being in this high-risk category, meaning they are most likely to convert to Parkinson’s disease based on their current symptoms.

## Implication
By focusing on the top 20% of at-risk patients, healthcare providers can prioritize these individuals for more frequent monitoring, early interventions, or tailored treatment plans. Identifying high-risk patients early can help slow disease progression or improve patient outcomes.


In [ ]:
y_train

## Neural Network

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler


# Step 4: Normalize the data using StandardScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 5: Define the neural network
model = Sequential()

# Input layer
model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

## Time Series analysis

In [ ]:
visit_mapping = {
    'BL': 1,
    'V02': 2,
    'V04': 4,
    'V06': 6,
    'V08': 8,
    'V10': 10
}

# Apply this mapping to the "Visit ID" column
df['Visit Number'] = df['Visit ID'].map(visit_mapping)

# Sorting data by Participant ID and Visit Number to ensure correct time sequence
df = df.sort_values(by=['Participant ID', 'Visit Number'])

In [ ]:
# Selecting relevant columns for the analysis
symptoms_columns = ['Tremor Mean', 'Rigidity Mean', 'Motor Function Mean',
                    'Posture and Gait Mean', 'Non-Motor Symptoms Mean']

# Grouping the data by Visit Number to calculate the mean of symptoms across visits
symptoms_progression = df.groupby('Visit Number')[symptoms_columns].mean()

# Function to forecast future visits using ARIMA model and plot results
def forecast_symptom(symptom_name, steps=6):
    # Selecting the symptom data and dropping null values
    symptom_data = symptoms_progression[symptom_name].dropna()

    # Define the ARIMA model (p=1, d=1, q=1) as an initial guess
    arima_model = ARIMA(symptom_data, order=(1, 1, 1))
    arima_result = arima_model.fit()

    # Forecast for the next `steps` periods (including V11 to V18)
    forecast = arima_result.forecast(steps=steps)

    # Plotting the historical data and forecasted data
    plt.figure(figsize=(10, 6))
    plt.plot(symptoms_progression.index, symptom_data, label=f'{symptom_name} (Historical)', marker='o')
    plt.plot(range(symptoms_progression.index[-1] + 1, symptoms_progression.index[-1] + 1 + steps),
             forecast, label=f'{symptom_name} (Forecast)', linestyle='--', marker='x')
    plt.title(f'Forecast of {symptom_name} Progression')
    plt.xlabel('Visit Number')
    plt.ylabel(f'{symptom_name} Value')
    plt.legend(loc='best')
    plt.grid(True)
    plt.show()

# Apply the forecasting function to "Tremor Mean"
forecast_symptom('Tremor Mean')

# Apply the forecasting function to "Rigidity Mean"
forecast_symptom('Rigidity Mean')

# Apply the forecasting function to "Motor Function Mean"
forecast_symptom('Motor Function Mean')

## Gradient Descent and Multiple Regression

In [ ]:
# First step labeling Parkinson or Healthy Control
# Creating the 'Parkinson' column based on the "Decoded Value for COHORT"
df['Parkinson'] = df['Decoded Value for COHORT'].apply(lambda x: 1 if x == "Parkinson's Disease" else 0)

# Checking the distribution of the new Parkinson column
df['Parkinson'].value_counts()
X = df[['Rigidity Mean','Tremor Mean','Motor Function Mean','Posture and Gait Mean','Non-Motor Symptoms Mean']].dropna()
y = df['Parkinson'].dropna()

In [ ]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardizing the features for better performance in gradient descent
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Using SGDClassifier with logistic loss function to perform gradient descent
sgd_classifier = SGDClassifier(loss='log_loss', max_iter=1000, learning_rate='optimal', random_state=42)
sgd_classifier.fit(X_train_scaled, y_train)

# Getting the weights (coefficients) learned by the model
sgd_weights = sgd_classifier.coef_

# Displaying the resulting weights
sgd_weights, sgd_classifier.intercept_

In [ ]:
def sigmoid(z):
    """
    Compute the sigmoid of z.

    Parameters:
    z: A scalar or numpy array of any size.

    Returns:
    The sigmoid of z.
    """
    return 1 / (1 + np.exp(-z))

In [ ]:
def cost_function(w1, w2, X1, X2, y):
    m = len(y)
    predictions = sigmoid(w1 * X1 + w2 * X2)
    cost = -(1/m) * np.sum(y * np.log(predictions) + (1 - y) * np.log(1 - predictions))
    return cost

# Creating a mesh grid for two variables (weights w1 and w2)
w1_vals = np.linspace(-1, 1, 100)
w2_vals = np.linspace(-1, 1, 100)
w1_vals, w2_vals = np.meshgrid(w1_vals, w2_vals)

# Assume we're using "Rigidity Sum" and "Tremor Sum" for the 3D visualization
X1 = X['Rigidity Mean'].values
X2 = X['Tremor Mean'].values
y_values = y.values

# Calculate the cost for each combination of w1 and w2
Z = np.array([cost_function(w1, w2, X1, X2, y_values) for w1, w2 in zip(np.ravel(w1_vals), np.ravel(w2_vals))])
Z = Z.reshape(w1_vals.shape)

# Gradient descent path for two weights (example)
path_w1 = np.array([0.5, 0.4, 0.3, 0.2, 0.1, 0.05, 0])
path_w2 = np.array([0.5, 0.45, 0.35, 0.25, 0.15, 0.1, 0.05])
path_cost = np.array([cost_function(w1, w2, X1, X2, y_values) for w1, w2 in zip(path_w1, path_w2)])

# Plotting the 3D surface for the cost function
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(w1_vals, w2_vals, Z, cmap='viridis', alpha=0.6)
ax.plot(path_w1, path_w2, path_cost, color='r', marker='o', label='Gradient Descent Path')

# Labels and titles
ax.set_title('Gradient Descent Path on Cost Surface')
ax.set_xlabel('Weight 1 (Rigidity Mean)')
ax.set_ylabel('Weight 2 (Tremor Mean)')
ax.set_zlabel('Cost')
plt.legend()
plt.show()

In [ ]:
gradient_weights = np.array([ 0.10789418,  0.03946231, -0.1268167 , -0.06324631,  0.77659627])
intercept = -0.08475943  # Intercept (bias term)

# Features (X) - the same features used in gradient descent
X_features = X[['Rigidity Mean', 'Tremor Mean', 'Motor Function Mean', 'Posture and Gait Mean', 'Non-Motor Symptoms Mean']].values

# Linear regression prediction: y_pred = X * weights + intercept
y_pred = np.dot(X_features, gradient_weights) + intercept

# Since this is a logistic regression-like model, we can apply the sigmoid function to interpret the predictions as probabilities
y_pred_probabilities = sigmoid(y_pred)

# Thresholding to classify as Parkinson's (1) or not (0), based on probability >= 0.5
y_pred_class = (y_pred_probabilities >= 0.5).astype(int)

# Displaying a few predicted classes
y_pred_class[:10], y_pred_probabilities[:10]

## Evaluate the Model

In [ ]:
# Ground truth (actual labels)
y_true = y.values

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred_class)

# ROC Curve
fpr, tpr, thresholds = roc_curve(y_true, y_pred_probabilities)
roc_auc = roc_auc_score(y_true, y_pred_probabilities)

# Classification Report
class_report = classification_report(y_true, y_pred_class)



In [ ]:
# Convert classification report to a dictionary first, then create a DataFrame
class_report_dict = classification_report(y_true, y_pred_class, output_dict=True)

# Convert the dictionary to a pandas DataFrame
class_report_df = pd.DataFrame(class_report_dict).transpose()

# Display the DataFrame
print(class_report_df)

# Visualization : ROC Curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='orange', label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.grid(True)
plt.show()


## Key Insight
The model is much better at detecting Parkinson's diseas(class1)but it struggles to identify true negatives (class 0), leading to poor performance for "Not Parkinson" cases.
Precision is balanced between the classes, but the low recall for class 0 suggests that the model is highly biased toward predicting Parkinson's disease.
An AUC of 0.72 indicates decent discrimination between the two classes.

## Agglomerative Hierarchical Clustering:


In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering


X = df[['Rigidity Mean', 'Tremor Mean', 'Motor Function Mean', 'Posture and Gait Mean', 'Non-Motor Symptoms Mean']].dropna()

# Step 2: Standardize the Data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 3: Compute the linkage matrix for hierarchical clustering
linked = linkage(X_scaled, method='complete')

# Step 4: Plot the Dendrogram
plt.figure(figsize=(10, 7))
dendrogram(linked, truncate_mode='level', p=5, orientation='top', distance_sort='descending', show_leaf_counts=True)
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Patient Index')
plt.ylabel('Distance')
plt.show()

# Step 5: Perform Agglomerative Clustering
cluster = AgglomerativeClustering(n_clusters=4, linkage='single')
labels = cluster.fit_predict(X_scaled)

# Step 6: Add the cluster labels to the dataset
df['Cluster'] = labels

# Display first few rows with clusters
df.head()

In [ ]:
# pip install mglearn


In [ ]:
import mglearn
import matplotlib.pyplot as plt

mglearn.discrete_scatter(X_scaled[:,0], X_scaled[:, 1], labels)
#plt.legend("Cluster 0", "Cluster 1", "Cluster 2", loc = "best")
plt.title("Agglomerative Hierarchical Clustering Plot (Single Linkage)")
plt.show()

In [ ]:
from sklearn.cluster import DBSCAN
dbscan = DBSCAN()
clusters = dbscan.fit_predict(X_scaled)

plt.scatter(X_scaled[:, 0], X_scaled[:, 1], c = clusters, cmap = mglearn.cm2, s = 30)
plt.title("DBSCAN Plot")

In [ ]:
from scipy.cluster.hierarchy import fcluster

clusters = fcluster(linked, t=60, criterion='distance')

#  Filter to include only numeric columns
numeric_cols = X

# Add the cluster labels to the numeric data
numeric_cols['Cluster'] = df['Cluster']

# Calculate the mean for each cluster, ignoring non-numeric columns
cluster_means = numeric_cols.groupby('Cluster').mean()
cluster_means


Cluster 1:
Rigidity Sum: 7.49
Tremor Sum: 2.79
Motor Function Sum: 19.00
Posture and Gait Sum: 9.23
Non-Motor Symptoms Sum: 5.09

Cluster 2:
Rigidity Sum: 3.60
Tremor Sum: 1.39
Motor Function Sum: 9.33
Posture and Gait Sum: 2.04
Non-Motor Symptoms Sum: 4.44

Cluster 3:
Rigidity Sum: 6.39
Tremor Sum: 5.65
Motor Function Sum: 12.02
Posture and Gait Sum: 2.56
Non-Motor Symptoms Sum: 6.36

Cluster 4:
Rigidity Sum: 0.44
Tremor Sum: 0.68
Motor Function Sum: 1.33
Posture and Gait Sum: 0.58
Non-Motor Symptoms Sum: 4.59

Cluster 5:
Rigidity Sum: 1.76
Tremor Sum: 1.92
Motor Function Sum: 5.03
Posture and Gait Sum: 1.17
Non-Motor Symptoms Sum: 15.58

## Summary of Clusters:
Cluster 1: Moderate-to-severe motor dysfunction with notable posture and gait issues.

Cluster 2: Mild motor symptoms, with minimal posture and gait impairment.

Cluster 3: Severe tremor, moderate motor dysfunction, and higher non-motor symptoms.

Cluster 4: Very mild motor symptoms but moderate non-motor symptoms.

Cluster 5: Mild motor symptoms but very high non-motor complications.

In [ ]:
# Visualize the distribution of Tremor Sum across clusters
plt.figure(figsize=(10, 6))
sns.boxplot(x='Cluster', y='Tremor Mean', data=df)
plt.title('Distribution of Tremor Sum Across Clusters')
plt.show()

# Visualize the distribution of Non-Motor Symptoms Sum across clusters
plt.figure(figsize=(10, 6))
sns.boxplot(x='Cluster', y='Non-Motor Symptoms Mean', data=df)
plt.title('Distribution of Non-Motor Symptoms Sum Across Clusters')
plt.show()

# Visualize the distribution of Non-Motor Symptoms Sum across clusters
plt.figure(figsize=(10, 6))
sns.boxplot(x='Cluster', y='Rigidity Mean', data=df)
plt.title('Distribution of Rigidity Sum Across Clusters')
plt.show()

# Visualize the distribution of Non-Motor Symptoms Sum across clusters
plt.figure(figsize=(10, 6))
sns.boxplot(x='Cluster', y='Posture and Gait Mean', data=df)
plt.title('Distribution of Posture and Gait Sum Across Clusters')
plt.show()

# Visualize the distribution of Non-Motor Symptoms Sum across clusters
plt.figure(figsize=(10, 6))
sns.boxplot(x='Cluster', y='Motor Function Mean', data=df)
plt.title('Distribution of Motor Function Sum  Across Clusters')
plt.show()